In [167]:
from transformers import pipeline

nasa_smd_pipeline = pipeline("question-answering", model="nasa-impact/nasa-smd-ibm-v0.1")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at nasa-impact/nasa-smd-ibm-v0.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [168]:
astroBERT_pipeline = pipeline("question-answering", model="adsabs/astroBERT")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at adsabs/astroBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [169]:
from datasets import load_dataset

squad_datasets = load_dataset("squad")
squad_data = squad_datasets["train"][50000:70000]

In [170]:
from evaluate import load
squad_metric = load("squad")

In [125]:
# nasa_result = nasa_smd_pipeline([dict(context="KU's School of Business launched interdisciplinary management science graduate studies in operations research during Fall Semester 1965. The program provided the foundation for decision science applications supporting NASA Project Apollo Command Capsule Recovery Operations.", question="What NASA program benefited from the use of decision science applications?"),
#                                  dict(context="On October 21, 1959, Eisenhower approved the transfer of the Army's remaining space-related activities to NASA. On July 1, 1960, the Redstone Arsenal became NASA's George C. Marshall Space Flight Center, with von Braun as its first director. Development of the Saturn rocket family, which when mature, would finally give the US parity with the Soviets in terms of lifting capability, was thus transferred to NASA.", question="The Redstone Arsenal became the George C. Marshall Space Flight Center when?")])

# nasa_result

In [171]:
# Converts squad data to squad reference data format
def convert_to_squad_reference_format(dataset):
    reference_result = []
    for i in range(len(dataset["context"])):
        reference_item = {
            "id": dataset["id"][i],
            "answers": [{"text": ans, "answer_start": start} for ans, start in zip(dataset["answers"][i]["text"], dataset["answers"][i]["answer_start"])]
        }
        reference_result.append(reference_item)
    return reference_result


In [172]:
reference_data = convert_to_squad_reference_format(squad_data)
reference_data

[{'id': '570b9fa36b8089140040f9c1',
  'answers': [{'text': 'at a high altitude', 'answer_start': 302}]},
 {'id': '570b9fa36b8089140040f9c2',
  'answers': [{'text': 'balloon', 'answer_start': 364}]},
 {'id': '570b9fa36b8089140040f9c3',
  'answers': [{'text': 'Space telescopes', 'answer_start': 388}]},
 {'id': '570ba092ec8fbc190045ba86',
  'answers': [{'text': 'Near-infrared', 'answer_start': 0}]},
 {'id': '570ba092ec8fbc190045ba87',
  'answers': [{'text': 'water absorption', 'answer_start': 254}]},
 {'id': '570ba092ec8fbc190045ba88',
  'answers': [{'text': '1,050', 'answer_start': 365}]},
 {'id': '570ba092ec8fbc190045ba89',
  'answers': [{'text': '950', 'answer_start': 416}]},
 {'id': '570ba1b2ec8fbc190045ba8e',
  'answers': [{'text': 'Heat', 'answer_start': 0}]},
 {'id': '570ba1b2ec8fbc190045ba8f',
  'answers': [{'text': 'thermal radiation', 'answer_start': 137}]},
 {'id': '570ba1b2ec8fbc190045ba90',
  'answers': [{'text': 'the solar corona', 'answer_start': 608}]},
 {'id': '570ba1b2ec

In [173]:
len(reference_data)

15000

In [174]:
# Converts squad dataset to question-context format
def convert_to_question_context_format(dataset):
    prediction_result = []
    for i in range(len(dataset["context"])):
        item = {
            "context": dataset["context"][i],
            "question": dataset["question"][i],
            # "id": dataset["id"][i],
            # "title": dataset["title"][i],
            # "answers": [{"text": ans, "answer_start": start} for ans, start in zip(dataset["answers"][i]["text"], dataset["answers"][i]["answer_start"])]
        }
        prediction_result.append(item)
    return prediction_result

In [175]:
# Converts pipeline's answer to squad prediction format
def convert_to_prediction_format(prediction):
    prediction_result = []
    for item in prediction:
        prediction_item = {
            "prediction_text": item["answer"],
            "id": item["id"]
        }
        prediction_result.append(prediction_item)
    return prediction_result


In [176]:
def apply_pipeline(pipeline, data):
    data = convert_to_question_context_format(data)
    prediction = pipeline(data)
    for i,item in enumerate(prediction):
        item['id'] = squad_data['id'][i]
    prediction_result = convert_to_prediction_format(prediction)

    return squad_metric.compute(predictions=prediction_result, references=reference_data)

In [177]:
# Results from nasa-smd pipeline
nasa_smd_result = apply_pipeline(nasa_smd_pipeline, squad_data)
nasa_smd_result

{'exact_match': 0.12666666666666668, 'f1': 4.615891622185711}

In [178]:
astroBERT_result = apply_pipeline(astroBERT_pipeline, squad_data)
astroBERT_result

{'exact_match': 0.4533333333333333, 'f1': 4.698943783720202}